In [ ]:
key = #use yr api key













In [27]:
from openai import OpenAI
import time
client = OpenAI(api_key=key)


In [28]:
#phase 1 
patient_case = input("Create case: Please input injury/illness, location if applicable, and any other details. Being more specific will result in a more detailed patient history")

patient_case = f"Create a hypothetical patient with {patient_case}. Create a whole patient history with name, age, handedness, occupation, previous medical history, allergies, or any other details you see fit. Ensure the history is realistic but keep it concise."

casefile = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {"role": "system", "content": "You are a medical professor that is creating a case for medical students to practise their skills on. Present the case based on the following conditions."},
    {"role": "user", "content": patient_case}
  ]
)
casefile = casefile.choices[0].message.content
print(casefile)

Patient Profile:
Name: Mr. Jason Smith
Age: 41
Handedness: Right-handed
Occupation: Computer Programmer

Presenting Problem:
Mr. Smith presents with severe pain and swelling in his right wrist following a fall while snowboarding over the weekend. He is right-handed and is experiencing significant discomfort when using his computer keyboard, which is necessary in his occupation as a computer programmer.

Medical history:
Mr. Smith has a medical history of moderate asthma controlled with regular use of Seretide (Fluticasone propionate/Salmeterol) and acute episodes with Ventolin (Salbutamol). He had an appendectomy at age 22 and fractured his left ankle at age 35. He is a non-smoker but consumes socially alcohol during the weekends. His vaccinations are up to date.

Family history:
His father has Hypertension, and his mother has type 2 diabetes. His older sister is diagnosed with rheumatoid arthritis.

Allergies:
Mr. Smith has an allergy to penicillin which causes skin rashes. 

Addition

In [29]:
nurse = client.chat.completions.create(
  model="gpt-4",

  messages=[
    {"role": "system", "content": f"""You are a nurse that will introduce a patient to a student doctor, the patient has the following case: {casefile}. Only provide brief patient background and their chief complaint. Do not disclose their illness or injury. Provide only details a nurse will know from a short conversation with the patient"""},
    {"role": "user", "content": "Who is the next patient?"}
  ]
)
print(nurse.choices[0].message.content)

The next patient is Mr. Jason Smith. He's a 41-year-old right-handed computer programmer who enjoys outdoor activities like hiking and snowboarding. Over the weekend, he had a snowboarding accident and he has come in today because he's experiencing severe pain and swelling in his right wrist. The pain increases significantly when he uses a computer keyboard, which is an essential tool for his job. He has a history of asthma, had an appendectomy and a fractured left ankle, and is allergic to penicillin. Mr. Smith lives an active lifestyle with his wife and two kids, and he is currently under a lot of work-related stress due to a project deadline.


In [30]:
print("The patient has now entered the room\n")

patient = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": f"You are a patient with the following case: {casefile}. You are now speaking to the docter. React accordingly to his introductory statement. Do not disclose your injury or illness."},
    {"role": "user", "content": "Hi what brings you here today?"}
  ]
)

print(patient.choices[0].message.content)

The patient has now entered the room

Hi, Doctor. Thank you for seeing me. I'm experiencing severe pain and swelling in my right wrist. I had a fall while snowboarding over the weekend, and it's been getting worse since then. The pain is making it difficult for me to use my computer keyboard, which is crucial for my job as a computer programmer. Can you help me with this?


In [32]:
#Phase 2


while True:
    choice = int(input("Choose what to do \n1. Check physical symptoms\n2. Ask the patient some questions\n3. Perform a scan\n4. Conduct a test\n5. diagnose the patient\n6. Stop\nChoice:"))
    
    if choice and choice not in [2,5]:
        action = input('What do you want to do?')
        phy_check = client.chat.completions.create(
        model="gpt-4",
        messages=[
        {"role": "system", "content": f"There is a patient with the following case: {casefile}. You are a medical simulator that will simulate what will happen accurately if the doctor performs certain tests or tries to tries to look for certain observations on the patient. Reply with only the test results or observations. Do not make any evaluations or give a list of possible outcomes. Give a single observation or outcome that is most likely with the case ONLY, Do not give reasons. Speak in the past definite tense"},
        {"role": "user", "content": f"What will happen if the doctor did the following: {action}?"}
            ]
        )
        time.sleep(5) #async timer
        phys_check = phy_check.choices[0].message.content
        #update summary
        actions[action] = phys_check
        print(phys_check)
        
    if choice == 2:
        patient_qns = input("what question do you ask?")
        patient_reponse = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
        {"role": "system", "content": f"You are a patient with the following case: {casefile}. You are now speaking to the docter. React accordingly to his questions"},
        {"role": "user", "content": patient_qns}
            ]
        )
        qna[patient_qns] = patient_reponse.choices[0].message.content
        print(patient_reponse)
        
    if choice == 5:
        diagno = input("what is your diagnoses?")
        result = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
        {"role": "system", "content": f"You are a strict medical evaluator. You evaluate student doctor's diagnoses and ensure it is right. There is a patient with the following case: {casefile}. A student doctor has made the following tests. {actions} Evaluate if he is correct and made the right choices"},
        {"role": "user", "content": diagno}
            ]
        )
        time.sleep(5)
        result = result.choices[0].message.content
        break

    if choice == 6:
        break
    
    choice = 6


        

The X-ray showed a fracture in the right wrist.
The x-ray of the left hand showed no abnormalities or signs of fracture.


In [33]:
result

"Based on the information provided, the student doctor's diagnosis is correct. The test results indicate that Mr. Smith has a fracture in his right wrist, which is consistent with his presenting problem of severe pain and swelling following a fall while snowboarding. The observation of significant swelling in the right wrist further supports this diagnosis. Additionally, performing an x-ray on the left hand and finding no abnormalities or signs of fracture helps rule out any injury on the non-dominant side."

In [34]:
#compare
acts = ','.join(list(actions.values()))
results = ','.join(list(actions.keys()))
anses = ','.join(patient_reponse) if patient_reponse else ''

final = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {"role": "system", "content": f"You are a medical professor accessing a medical student. This is the full case file: {casefile}. The student will give his diagnoses and his process in obtaining it. Focus on assessing his methods."},
    {"role": "user", "content": f"I have done {actions} and the results were {results}. From the questions and answers i got: {anses} thus I believe the diagnoses is {diagno}"}
  ]
)

In [35]:
final.choices[0].message.content

"The medical student's diagnosis seems logical based on the information in Mr. Smith's case file, and available test results. Their approach to diagnose was by a physical examination of the wrist for swelling, which is a common symptom of fractures, and performing an X-ray on the right hand. The X-ray, which is the standard test to confirm a fracture, revealed a fractured wrist as suspected. The student also took an extra precaution by checking the left-hand with an X-ray to rule out any unnoticed injuries. \n\nThe student took a systematic approach: symptom evaluation, followed by confirmation with a specific test. The student further followed the principle of 'comparison' in diagnostic imaging by having the contralateral side (the left wrist in this case) as reference for what's normal. \n\nHowever, in a real-world setting, the X-ray of left hand could have been avoided initially unless there was a complaint of pain, or suspicion arose from physical examination. Apart from this, the 

1. Generate image for the xray
2. Make the doctor types all the response
3. Treatment plans 

In [36]:
actions

{'check for swelling': "The doctor observed significant swelling in Mr. Smith's right wrist.",
 'perform an xray on the right hand': 'The X-ray showed a fracture in the right wrist.',
 'Perform an xray on the left hand': 'The x-ray of the left hand showed no abnormalities or signs of fracture.'}